In [1]:
# 현재 구동되고 있는 MySQL 서버 프로세스를 확인하는 방법
# ps -ef / grep mysql

# 쓸모없는 정보가 같이 나오는데 걸러내는 방법
# ps -ef

# netstat 명령어를 사용하기 위해 필요한 SW툴
# sudo apt-get install net-tools

# 현재 구동중인 네트워크 서비스들을 확인해볼 수 있다.
# sudo netstat -ntlp | grep mysql

# 127.0.0.1:3306
# 일반적으로 127.0.0.1

In [2]:
# pip install pymysql
import pymysql

# mysql -u bitai -p 비밀번호
# 위의 명령을 프로그래밍 버전으로 변형한 것임
db = pymysql.connect(
    # 접속할 컴퓨터의 주소
    host = '127.0.0.1',
    # 현재 구동중인 MySQL의 포트(서비스) 번호
    port = 3306,
    # 계정 이름
    user = 'bitai',
    # 비밀 번호
    passwd = '456123',
    # 사용하려는 스키마
    db = 'pydb'
)

# 후에 select 등을 했을때
# 결과를 하나 하나 커서를 내려가며 순회할 수 있는 녀석
cursor = db.cursor()

# 실제 MySQL에서 사용하는 DB Query 문임
# create table 테이블명(원하는 필드와 원하는 타입을 작성)
# auto_increment는 자동 증가
# unsigned란 부호가 없다는 의미(즉 양수만 취급)
# varchar는 문자를 저장할 수 있는 배열
# primary key는 대표 키를 설정하는 키워드
sql = """
    create table pydbtest(
        id int unsigned not null auto_increment,
        name varchar(20) not null,
        price int not null,
        primary key(id)
    )
"""

# 위의 SQL Query를 실행하는 코드
# 작업을 메모리 상에서 해놓고 메모리에 올려놓고 있음
cursor.execute(sql)

# DB에 실제 내용을 갱신하는 코드
# 실제 이부분은 I/O로 작업을 내리는 과정
db.commit()

# 왜 실행하자마자 적용되지 않고 commit()이란 것이 별도로 있나요 ?
# 파이썬에서 join()을 배웠었는데 같은 이유다.
# 결론적으로 컴퓨터의 성능은 I/O와 직결된다.

# 작업을 완료했으므로 DB 연결을 끊는다.
db.close()

print("Table Creation Success")

Table Creation Success


In [3]:
# 위의 코드를 통해서 pydb 스키마에
# pydbtest라는 테이블을 생성하였다.
# 확인은 아래와 같이 하면 된다.
# mysql -u bitai -p

# MySQL에 접속해서
# use pydb;
# show tables;
# 위의 명령을 수행하면 방금만든 pydbtest 테이블을 볼 수 있을 것이다.

# 내부의 내용을 살펴보고자 한다면
# describe pydbtest;

In [8]:
import pymysql

db = pymysql.connect(
    # 접속할 컴퓨터의 주소
    host = '127.0.0.1',
    # 현재 구동중인 MySQL의 포트(서비스) 번호
    port = 3306,
    # 계정 이름
    user = 'bitai',
    # 비밀 번호
    passwd = '456123',
    # 사용하려는 스키마
    db = 'pydb'
)

cursor = db.cursor()

# insert into 테이블이름(필드 이름들 채움)
# 필드를 이름을 채울때 id처럼 auto_increment 되는 것을 제외한다.
# values(%s, %s)라고 되어 있는데
# 이부분은 어떤 특정값을 가져와서 문자열 형태로 처리할 것이라는 의미
sql = """
    insert into pydbtest(name, price) values (%s, %s)
"""

# 위의 %s가 두개 있으므로
# sql로 전달되는 인자 또한 두개가 들어온다.
# execute할 때 sql의 %s에 잉ㄴ자로 넘온 튜플이 들어가게된다.
cursor.execute(sql, ('Xeon Server', 10000000))
cursor.execute(sql, ('FFGA Server', 34000000))
cursor.execute(sql, ('GPU Server', 30000000))
cursor.execute(sql, ('RTX Titan Server', 40000000))

db.commit()

db.close()

print("Table Creation Success")

Table Creation Success


In [10]:
import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

cursor = db.cursor()

# pydbtest에 있는 모든것(*)을 선택해서 가져와라~
sql = """
    select * from pydbtest;
"""
cursor.execute(sql)
rows = cursor.fetchall()
print(rows)

db.close()

print("Selection Success")

((1, 'Xeon Server', 10000000), (2, 'FFGA Server', 34000000), (3, 'GPU Server', 30000000), (4, 'RTX Titan Server', 40000000))
Selection Success


In [13]:
import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

cursor = db.cursor(pymysql.cursors.DictCursor)


sql = """
    select * from pydbtest;
"""
cursor.execute(sql)
rows = cursor.fetchall()
for row in rows:
    print(row['id'], row['name'],row['price'])

db.close()

print("Selection Success")

1 Xeon Server 10000000
2 FFGA Server 34000000
3 GPU Server 30000000
4 RTX Titan Server 40000000
Selection Success


In [14]:
import pymysql

db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'bitai',
    passwd = '456123',
    db = 'pydb'
)

cursor = db.cursor(pymysql.cursors.DictCursor)

# 기존에 만든 테이블을 삭제하는 방법
# drop table 테이블이름
sql = """
    drop table pydbtest;
"""
cursor.execute(sql)

db.commit()

db.close()

print("drop Success")

drop Success


In [15]:
import time
import math
import pymysql
import calendar
import datetime
import numpy as np

In [19]:
sampleTime = np.arange(0, 10, 0.001)
noiseData = np.random.normal(size = len(sampleTime))

voltage = 3 * np.cos(10 * sampleTime + (math.pi / 3)) + \
    2.7 * np.cos(7 * sampleTime) + \
    2.5 * np.cos(5 * noiseData * sampleTime)

current = 3 * np.cos(10 * sampleTime) + \
    2.7 * np.cos(7 * sampleTime) + \
    2.5 * np.cos(5 * noiseData * sampleTime)  


In [20]:
ts = calendar.timegm(time.gmtime())
ts = (sampleTime * 1000) + ts

sampleTime = np.array(sampleTime, dtype=np.float32)
voltage = np.array(voltage, dtype=np.float32)
current = np.array(current, dtype=np.float32)

In [22]:
currTime = []

for i in range(len(sampleTime)):
    currTime.append(
        datetime.
            datetime.
                fromtimestamp(ts[i]).
                    strftime('%Y-%m-%d %H:%M:%S')
    )